In [ ]:
import matplotlib.pyplot as plt

def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    fig, axes = plt.subplots(n_row, n_col, figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    
    n_images = n_row * n_col
    for i, ax in enumerate(axes.ravel()[:n_images]):
        ax.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        ax.set_title(titles[i], size=12)
        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout()
    plt.show()


In [ ]:
import os
import cv2
import numpy as np

dir_name = "dataset/"
y = []
X = []
target_names = []
class_names = []
person_id = 0
h = w = 300  # image height and width
n_samples = 0

# Loop over the directory containing images of each person
for person_name in os.listdir(dir_name):
    person_dir = os.path.join(dir_name, person_name)
    
    if os.path.isdir(person_dir):  # Ensure it's a directory
        class_names.append(person_name)
        
        # Loop over each image in the person's directory
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            img = cv2.imread(image_path)

            # Check if the image was successfully loaded
            if img is not None:
                # Convert the image to grayscale
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                # Resize the image to the target dimensions (300x300)
                resized_image = cv2.resize(gray, (h, w))
                # Flatten the image to a vector
                X.append(resized_image.flatten())
                # Append the label
                y.append(person_id)
                # Increment sample count
                n_samples += 1

        # Increment person ID for the next class
        person_id += 1

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)
target_names = np.array(class_names)

# Check the shape of the dataset
n_features = X.shape[1]
n_classes = target_names.shape[0]

# Print summary statistics
print(f"y shape: {y.shape}, X shape: {X.shape}, target_names shape: {target_names.shape}")
print(f"Number of samples: {n_samples}")
print("Total dataset size:")
print(f"n_samples: {n_samples}")
print(f"n_features: {n_features}")
print(f"n_classes: {n_classes}")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt

# #############################################################################
# Split into a training set and a test set using stratified K-Fold
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

# #############################################################################
# PCA for dimensionality reduction (eigenfaces)
n_components = 150
print(f"Extracting the top {n_components} eigenfaces from {X_train.shape[0]} faces")

# Apply PCA and compute eigenfaces
pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Eigenfaces for visualization
eigenfaces = pca.components_.reshape((n_components, h, w))

# Plot the top eigenfaces
eigenface_titles = [f"Eigenface {i}" for i in range(n_components)]
plot_gallery(eigenfaces, eigenface_titles, h, w)
plt.show()

# Print PCA-projected data shape
print(f"Training data after PCA: {X_train_pca.shape}, Test data after PCA: {X_test_pca.shape}")

# #############################################################################
# LDA for further dimensionality reduction (Fisherfaces)
lda = LinearDiscriminantAnalysis()

# Fit LDA on the PCA-reduced data and transform both training and testing data
X_train_lda = lda.fit_transform(X_train_pca, y_train)
X_test_lda = lda.transform(X_test_pca)

# Print LDA-projected data shape
print(f"Training data after LDA: {X_train_lda.shape}, Test data after LDA: {X_test_lda.shape}")

print("Projection done...")


In [ ]:
from sklearn.neural_network import MLPClassifier

# Training with Multi-layer Perceptron (MLP)
clf = MLPClassifier(random_state=1, hidden_layer_sizes=(10, 10), max_iter=1000, verbose=True)

# Fit the model on the LDA-reduced data
clf.fit(X_train_lda, y_train)

# Output basic model information
print("Training complete.")
print("Model Weights (Shape of each layer's weights):")
model_info = [coef.shape for coef in clf.coefs_]
print(model_info)

# Optional: If needed, you can also output the full weights (but this can be verbose):
# print("Full model weights:", clf.coefs_)


In [ ]:
from sklearn.metrics import accuracy_score

# #############################################################################
# Predict class labels and probabilities
y_pred = clf.predict(X_test_lda)
y_prob = clf.predict_proba(X_test_lda).max(axis=1)  # Max probability for each prediction

# #############################################################################
# Compute Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# #############################################################################
# Prepare titles for plotting
prediction_titles = []
for i in range(len(y_pred)):
    true_name = class_names[y_test[i]]
    pred_name = class_names[y_pred[i]]
    prob = f"{y_prob[i]:.2f}"
    
    result = f"pred: {pred_name}, pr: {prob}\ntrue: {true_name}"
    prediction_titles.append(result)

# #############################################################################
# Plot the gallery of test results
plot_gallery(X_test, prediction_titles, h, w)
plt.show()
